In [ ]:
import requests
import lxml.html
import sqlite3
from pandas.io import sql
import os
import re
import string
import pandas as pd
from tabulate import tabulate
from selenium.webdriver import Chrome
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from selenium import webdriver
from selenium.webdriver import ActionChains
import csv

In [ ]:
def get_urls():
    
    page = 1
    max_page = 0
    
    response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do?A=A&searchType=all&searchPageRowCnt=50&pageIndex=1')
    root = lxml.html.fromstring(response.content)
        
    # 페이지 번호 중에서 max 페이지 가져오기    
    max_page = int(root.xpath('//*[@id="contents"]/div[4]/div[1]/a[13]/text()')[0])
    
    while(True):
        url_list = []

        response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do?A=A&searchType=all&searchPageRowCnt=50&pageIndex={}'\
                               .format(page))
        root = lxml.html.fromstring(response.content)
        for tr in root.xpath('//*[@id="contents"]/table/tbody/tr') :
            a = tr.xpath('td[3]/a')[0]
            url = 'https://world.moleg.go.kr' + a.get('href')
            url_list.append(url)

        if (page >= max_page):
            break
        else:
            page = page + 1

        time.sleep(1)
        
    return url_list

In [ ]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△《》•‘’○※▷【】]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string2 = re.sub('\n', ' ', new_string) # newline 제거
    new_string3 = re.sub('  ', ' ', new_string2) # double space 제거
    new_string4 = re.sub('   ', ' ', new_string3) # double space 제거
    return new_string4

In [ ]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [ ]:
def get_info(url):
    details = []
    
    browser.get(url)
    title = browser.find_element('xpath','//*[@id="contents"]/div[3]').text
    country = browser.find_element('xpath','//*[@id="contents"]/div[2]/div[2]/strong').text
    upload_date = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[1]/span').text
    views = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[2]/span').text
    trs = browser.find_element('xpath','//*[@id="lgslTrend_CTS"]').text
    
    browser.implicitly_wait(4)
    
    words = remove_punc(trs)
    details.append(title)
    details.append(country)
    details.append(upload_date)
    details.append(views)
    details.append(words)
    
    return details

In [ ]:
def db_save_as_csv(data):
    data.to_csv("법제동향_page1.csv", index=False, encoding='utf-8')

In [ ]:
from selenium.webdriver.chrome.options import Options

options = Options()

#바이너리 위치 설정
#options.binary_location= 'C:\Program Files\Google\Chrome\Application\chrome.exe'

urls = get_urls()
df_list = []

browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정

for url in urls:
    
    details = get_info(url)
    
    df = pd.DataFrame([{
        "제목": details[0],
        "국가": details[1],
        "등록일자": details[2],
        "조회수": details[3],
        "본문내용": details[4]
    }])
    
    df_list.append(df)
    
    browser.implicitly_wait(4)

ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)
db_save_as_csv(ARTICLE_LIST)

browser.close()
browser.quit()

In [ ]:
with open(r"법제동향_page1.csv", 'r', encoding='utf-8') as f:
    
    document = pd.read_csv(f)
    
    display(document)